In [6]:
import psycopg2
import requests
import json
import pandas as pd
import time

In [7]:
MAAS_ENDPOINT = "https://model-service.worldmodelers.com/"
#conn = psycopg2.connect(host="maas.cowfye1azh5f.us-east-1.rds.amazonaws.com",database="output", user="wmuser", password="REPLACEME")
#cur = conn.cursor()
user = "wmuser"
password = "Bq248mQHV9EDuUap"

In [18]:
# get all training data (configs + outputs)
def get_training_data(model_name):
    response = requests.get(MAAS_ENDPOINT + "list_runs/" + model_name)
    run_ids = json.loads(response.text)
    
    datas = []
    configs = []
    for rid in run_ids:
        #rid = rid.lower()
        config_response = requests.get(MAAS_ENDPOINT + "run_results/" + rid)
        config = json.loads(config_response.text)        
        statement = f"SELECT * FROM rundata where run_id = '{rid}';"
        cur.execute(statement)
        print(cur.fetchall())
        output_df = pd.DataFrame(cur.fetchall(),columns=['run_id', 'x', 'y','id','feature','datetime'])
        datas.append(output_df)
        configs.append(config)
            
    return configs, datas

In [70]:
np.arange(.0000000011,1.25,.24999999999)

array([  1.10000000e-09,   2.50000001e-01,   5.00000001e-01,
         7.50000001e-01,   1.00000000e+00])

In [3]:
"""
{
   "config":{
      "_type": 'mm_data',
      "dekad": '01',
      "year": 2019
   },
   "name":"CHIRPS"
}
"""

import pandas as pd
import numpy as np
import itertools
d = pd.read_csv("/Users/jgawrilow/single_shock_results.csv")
countries = list(d.country)
#countries = ["ETH"]
_type = ["mm_data"]
dekad = map(lambda x: str(x).zfill(2),range(1,37))
year = list(range(2019,1980,-1))
mmatrix = list(itertools.product(*[_type,year,dekad]))
print(mmatrix)

[('mm_data', 2019, '01'), ('mm_data', 2019, '02'), ('mm_data', 2019, '03'), ('mm_data', 2019, '04'), ('mm_data', 2019, '05'), ('mm_data', 2019, '06'), ('mm_data', 2019, '07'), ('mm_data', 2019, '08'), ('mm_data', 2019, '09'), ('mm_data', 2019, '10'), ('mm_data', 2019, '11'), ('mm_data', 2019, '12'), ('mm_data', 2019, '13'), ('mm_data', 2019, '14'), ('mm_data', 2019, '15'), ('mm_data', 2019, '16'), ('mm_data', 2019, '17'), ('mm_data', 2019, '18'), ('mm_data', 2019, '19'), ('mm_data', 2019, '20'), ('mm_data', 2019, '21'), ('mm_data', 2019, '22'), ('mm_data', 2019, '23'), ('mm_data', 2019, '24'), ('mm_data', 2019, '25'), ('mm_data', 2019, '26'), ('mm_data', 2019, '27'), ('mm_data', 2019, '28'), ('mm_data', 2019, '29'), ('mm_data', 2019, '30'), ('mm_data', 2019, '31'), ('mm_data', 2019, '32'), ('mm_data', 2019, '33'), ('mm_data', 2019, '34'), ('mm_data', 2019, '35'), ('mm_data', 2019, '36'), ('mm_data', 2018, '01'), ('mm_data', 2018, '02'), ('mm_data', 2018, '03'), ('mm_data', 2018, '04'),

In [6]:
import itertools
run_matrix = {
    "x":range(0,100),
    "y":["A","B","C"],
    "z":["Z","Y","X"]
}

test = [[1,2,3],[4,5,6]]

result = list(itertools.product(*test))
print(result)

[(1, 4), (1, 5), (1, 6), (2, 4), (2, 5), (2, 6), (3, 4), (3, 5), (3, 6)]


In [9]:
"""
if params are 
-- x float [0-100]
-- y A B C
-- z Z Y Z
"""

run_matrix = {
    "x":range(0,100),
    "y":["A","B","C"],
    "z":["Z","Y","X"]
}

def range_to_list(x):
    vals = []
    k = x.keys()[0]
    for v in x[k]:
        vals.append({k:v})
        
#result = list(itertools.product(*inputdata))
    

# continuously run model and cache outputs
def run_and_cache(session,model_name,runs,fields):
    
    for j, run in enumerate(runs):
        mydata = {
           "config":{},
           "name":model_name
        }
        for i, field in enumerate(fields):
            mydata["config"][field] = run[i]

        URL = MAAS_ENDPOINT + "run_model"
        #print(json.dumps(mydata))
        resp = session.post(URL, json=mydata)
        rid = resp.text.replace('"',"").strip()
        time.sleep(2)
        run_response = session.get(MAAS_ENDPOINT + "run_status/" + rid)
        STATUS = run_response.text.replace('"',"").strip()
        print(j, rid, STATUS)

        while STATUS != "SUCCESS" and STATUS != "FAIL":
            time.sleep(1)
            URL = MAAS_ENDPOINT + "run_status/" + rid
            run_response = session.get(MAAS_ENDPOINT + "run_status/" + rid)
            STATUS = run_response.text.replace('"',"").strip()
            print(j, rid, STATUS)
        
        
        
    

In [25]:
configs, datas = get_training_data("DSSAT")
print(datas)

NameError: name 'get_training_data' is not defined

In [12]:
session = requests.Session()
session.auth = (user, password)
run_and_cache(session,"CHIRPS",mmatrix,["_type","year","dekad"])

0 d905227ebee952cafdafdc49e3dec2356754a2b36983064075d1aca732292d9d SUCCESS
1 7c2755930ff8962208a4d4878d18d8f92a2cc1ec80f56b1d6a799c7875f6cb2f SUCCESS
2 3f1c1ab7a96747b9f325ec651691e04d14e8bcbb4ea91e2ac98c187b1325dc43 SUCCESS
3 22cd0efeee5a9c29104a90c5ae96b0f715b53628d223dea3848a54bfbf18f50d SUCCESS
4 8c6488f407f2d0b0aa117568b44aa5a77c36351c2cced3b271140283ae9a0eb3 SUCCESS
5 0f2dd8d2e73821c0bdb6d77627f133c867045fc098bd06d92b008b55274ccf02 SUCCESS
6 b5c68ba45ebf9baad0b341bf760b73242564e077e134816e0fc8cebcb408a005 SUCCESS
7 9e52cfec416300f25374483a064a749c5a5b3cc6421c4e111b7ab65c70a66491 SUCCESS
8 21da04a6cb6218dd9762b4e2aed5b92c196318a39010ab8c52881cc23615f2bc SUCCESS
9 34b4acedc2bcc4ef90cf1f482bbe15156824dd865992723a11ce9a6c68d0b528 SUCCESS
10 2a86cae1387e23d0dc85eae5b5b307767d4caf41df99aedbc46c43e583a6e13d SUCCESS
11 f1a2fbaab86a0c0761843fffce741c68a938c893fcfa35b53b70130e10db8daa SUCCESS
12 ca0c7444ee09d2d86e07060a6f2626e6d5be9d0b60c07a0fd742cd6d985d44c2 SUCCESS
13 c30cc8dd0dc6a14f7c5

108 f1d0db00ce96470975467abca637a8911fa322452f214afb177e78d1fdbde239 SUCCESS
109 f1fe5b69adbb80f4327fe6ffe00436628a05ad7b185d1a836b16bc117d4257c6 SUCCESS
110 a4bc018b62033815b6bf5bf1beb8ef04c6cf72c43fc82d4a14a7427ff493ac30 SUCCESS
111 04a1cb21f356d7c166d3fbedddf5d1e2a9af2523d4ceea42b621638f40746ee1 SUCCESS
112 d9364499297a8dd9519d63a6a40959f4c21cfa5fcf6d1a11a3c791930b06cfc2 SUCCESS
113 10174b16f299f6b9ca3e57094bf0a2ec7d70def47301c2450d6464c601a90cff SUCCESS
114 7464e567606cb8d33ac3205378bba8777c2f158c6e0f6eec06deb0ba0a7c9100 SUCCESS
115 b52832325a49646d3a3ce89f3ac1123b8fa0bd9718fbdcadf82ad76415ff4018 SUCCESS
116 d689d8567160c6a776c4331c7e9a936690b19bbc967c72b2791ae953dd3c441a SUCCESS
117 8af75700b95559a46943f5be243bb3096ccc6c0b7af0f44ba2aef12345e4bb3b SUCCESS
118 10b716387e14649a50d8437e754e4bffd6199d5075beae0b807b6ee84ed4fdb6 SUCCESS
119 f38cf522716d1f3d4ef7b152479d285c830ffe2c3033bc185014ad7acab98760 SUCCESS
120 fbc6d8b91191aaa77f2a79468aa6c0d507c6a7e99964880cc903b775051ca7ea SUCCESS

215 318bd77f5e4ccafb2c9425fd04d44a9d24cca8f2ff5db65ad4304a24d79e1a90 SUCCESS
216 d82d7a0d8553905831ff21d622b11fa573ea044cde205351211ac5027a0adf28 SUCCESS
217 cf3bf9b7208c142d066a9148429501706b0f01eefad006989b460d68526026d2 SUCCESS
218 8b3617196ed4ab8e8f39528be0779da721681cf171848d2978b62001a72f6cba SUCCESS
219 78be2351bf4a78484d5c3e2c2fc544ceb9d6c8fd042990449350b554d852d4f0 SUCCESS
220 ffdc7874c7959ebc372ce2fc0b948b47051eca45bca5be760e4d546b9bfaf86a SUCCESS
221 a11ebc9f617f59b0d9ac8da1cf09d311b99b7c22d20c148be721188f23b90579 SUCCESS
222 9c164adace3ddbf8b7e86e4ee1db932ace5b1b5bd439b0d33d5b1431f9a72eb1 SUCCESS
223 ab43ace62f11f7de4624efca7f45e2da1a03442925a84a14511efb9d8a5aaeaf SUCCESS
224 3e46077b0fc103ad321eae74985779bbf8ecbf10bcab7d3f4af021deb96c60ab SUCCESS
225 0eb75ef7c11cee5812fa4830dfe3e5035277d072d6e97aa31e2f739de951d991 SUCCESS
226 7b4ade5a75d772a50e43dc97427f0c92195043fd48d3668f2d9959fafbbe5760 SUCCESS
227 6afeabcdac224d30b9ac7752dac3e157b121abba95acab01f71f998d645495b7 SUCCESS

322 c9f3fde61ff84a1e005d95e9a1bbf3301982542a7ed9cc8bc9c0b4e3396280a2 SUCCESS
323 56c118e1122130443edbeb08d6561db53d37df2a7dfbdd5227de7bc4c1c5283f SUCCESS
324 dacb1de3a95e5b6607cf2f3f058a69ef539b2b5ac5f8446ec8f00995181a3f1a SUCCESS
325 6cba353e803c79696ac6ccc0f60d8d06884d8a49773ea9e2dead82b9d3262d45 SUCCESS
326 7adc1ba77d1b013305f2efa0db17a774de4be71ad8e468c7f104bd5642338794 SUCCESS
327 b383a3facac561c0eb72e804764ea19c703c75bff51282da7e56ffa1891c94f9 SUCCESS
328 5b37938e0b96845e1346af0491ade3391c9d0a01caf0412278e5a2d7637290ba SUCCESS
329 9ba44ee7344aa90dea529972781f1209c4db2c04a5ec84fff470b852e9f064fe SUCCESS
330 4b66d373704e2cfe8726b92f31dc59138398c3654515b3aa503d5d2d38a986c0 SUCCESS
331 b7f92d6af4c765e369383262a68a1622b49d9a9a4d1df2f0d7e3112f1a7f270a SUCCESS
332 a69a2226e42f2a904bde7106d72db4d23115963c174f06c87b1d0bcbe6619cca SUCCESS
333 0ca38bdc919ca8a363cf9ee10406d50156b4107873821bc4c74caafb527e191d SUCCESS
334 bd0d60ea48e6f879883f54bf1ccf80d24cb5a25bc1abf8cf6dcf0afda25fc283 SUCCESS

429 cb88370da778385f9f1614df22f4a52887ad1b3777ffc1834902b033dfcef9a9 SUCCESS
430 c63fa2fbeb0464abcb2ad8cdd26e1b0133e0a7a69fcb07bc315e347a7da85917 SUCCESS
431 d3b752b5e5dd40aad35c765808e17a61dc19184b425e1d2cd164f2d3fbeb4697 SUCCESS
432 f0c7566570aa3de12f33fda77b80be7898a7574ef15cb395f576f28a09e90048 SUCCESS
433 92f1c43af3eda23fb56b85d96b769d2d5621ccc632e1c715504f564cf46a248a SUCCESS
434 f84a004ea99344b7b6e245f10db6aa487b5a3772ba2c998b983b57294125328d SUCCESS
435 c02a4b2de753075725ca01a027a5c2136e3c4ad638909b34976aaecac3f337ec SUCCESS
436 0dcdd8a145990262ff2365161afca3864eed9aea17a798659ba5c7ca41b183e5 SUCCESS
437 7197fe476552004cf6022972149d2aaff54f0f0ec54c7e76a9080de2a906aa59 SUCCESS
438 c7c328034a692ea02858c3a96a9a0d62a529c99b0a1276998e54df635ce2a464 SUCCESS
439 b1092510bf3bdc7e187e15620d242040950f56acd09ec63441f20dcdbda990e3 SUCCESS
440 fa0c10226c3e63e15ffd2254f5ce172eb4b7a501305b232d15296f2b720c01fa SUCCESS
441 c3e9e8e58f4b490f309eb56f251addf0e86aa9916d18f36dc66a10103f5f717d SUCCESS

...

964 1a92ddb57618afeb1000d5e80d1b6c6e0886854d13844f4e35fac57a2030e137 SUCCESS
965 028581fe38fffeb7665233583484bbfc4c1e01bd2298001d5aad1a13c68bfad9 SUCCESS
966 f29ae120f245d80ef0487bffb7a9340e0937842a40c715a294a35f1d5202ea77 SUCCESS
967 c5c07a8d305b429a41f5533a44bafb43c6df781d09b17bd3346b5ffbe978518f SUCCESS
968 d6263b24ab668ffe9876d69b634c716da502102c3616028acff69e8e194fa9c3 SUCCESS
969 ef78c5d4fe0b790c34472c46574be62ee911a899733e54b6a0bd08e6eea9585f SUCCESS
970 bb9d4f2ac7a4d7ce40d49f746630c8033c49f6108ff8f7df5ec2f0ebd47652b8 SUCCESS
971 e42fcc74aad234529194568d8e2164522d0dd299e19b33d5638cae8fb2ab9b7f SUCCESS
972 4d79cefafcdfd657312b0c3b983b977effacd981ece444ba4c371a3458fc9d39 SUCCESS
973 b926613a91c6c09c48fb91b7fdb72304ebb4c1ee9790703fa27341887964d33c SUCCESS
974 879e6f2e79f55be00d17a337df1e023a042f4a993dd8a20a624f95f24935cb15 SUCCESS
975 c4a958fa95642a2d9288922c9a8c2f5a11afce77e4eb2c85d76b4d9650780c56 SUCCESS
976 b6210d5ea35cc3dca7fe4cd2669d000fa223b5ed1ddafb143bf1a43ef5940669 SUCCESS

1070 e3aadcfcb0f09a921e9210c5d19a24face3af7326b5f9806a0e18cd7c03d3484 SUCCESS
1071 0c2ea2c23bf634404c77eef5136a23e9dac477977f33772f0e6df0ac49098e7c SUCCESS
1072 27d6f56a99958d05730eb02c78a86d1fab25316f639364c1a78d9bde0e811fd6 SUCCESS
1073 9aa270e07066036ae2eebcb64bdfffc3ed701ec0838c89dfabd78de43aec3062 SUCCESS
1074 7657fb62e4229ea729f81b1f1a5e2999a56673bad8e07bdfbc2af8724a85c5dd SUCCESS
1075 c2d31b2262311dfed01687a1e1cb1e00c6ac0af21076c43ee76cee157cfbd84d SUCCESS
1076 8234ef8b93b56544b5d4f0fdd4224062f9b5ab8e6c69bad05ff9452d47aff31f SUCCESS
1077 47f905a6f49b5c66d57dda897d8e2b5aa427530a1eee3476fe1ac5f95c21cbbc SUCCESS
1078 919bf2d3a7aca69fbdf3adc84238fe317cc419ffa327f02543b0e9f8359f8ec5 SUCCESS
1079 ee11617c84d18439d132dc4eb6842250fa87cdc8ad821066d953b8177d663087 SUCCESS
1080 6f493ec13b103b7857b533af731583cbd258fa6b5c39fd2993bda38722956891 SUCCESS
1081 7b53a88a5e2784a6f00956a3d8855daf14886906ae46d4469e93213f6804ff4a SUCCESS
1082 297bfc2d0fa787dce82730a621182bbe44aec8641d59f368abc9d80ea9a

1176 15bc6e5ff939949de181c725587253f7aed04b9d67ce47275d1137a792aa6eeb SUCCESS
1177 d4c311324c2e4f6fdfef360bf59e46d17a02cc25348828993405de6ac858ef8c SUCCESS
1178 b4ab2df973d9614270663cc29fd9768a0f60b09311545137286c3ca0980f04d4 SUCCESS
1179 86a8d61c6666b55442950df605e422fd80c021dbbb70d5237c244381d2d32c6d SUCCESS
1180 6999bbe7c9e29d9473c135b51996760dd0f556e7e5322dd68f64f28d77ff345d SUCCESS
1181 d52496e6658d59837016ae5bc4fa0dbd9eb7fd1d356d88f6def5eed14ba3f0d0 SUCCESS
1182 e42a665412bcc6172171a4bbf8853b58ab6bc38879d6c5c56cd386f78a11e2fe SUCCESS
1183 b01e1913977a7fb4eb621a899eb5bf5eb62d758599aad4cff045d6deb65ec980 SUCCESS
1184 809ee489fb1d69ff13ae2f0e147cfa5edda96427f0bb3d00e829017db2b48137 SUCCESS
1185 0d91133a218c9ae39549974b6cbfd934a8101501111e2cde289501bcac8fecde SUCCESS
1186 58b8131e7aadc5fde288c40c67e0903de65e6fa222b2044ce61499977f02e538 SUCCESS
1187 6469e5de6c614735fbf8a786201367c15170f7b78c8018a41be9ecad1a99d185 SUCCESS
1188 7ab279a97d13a01ff6d01aa41b8ef8a274e09d2b5cd65f43b628900bc0a

1282 5d6d41679476883295d16226b9b046f9f15abc875478fea91c8b8d0718849ab9 SUCCESS
1283 b7994382c9f5d1ed2032dfb0307f82712ff16689074a65fa63a406d60c3ab941 SUCCESS
1284 ba5efca7faaf170c91151b27943006a1983b0a49f57b05de11ef8dff10cc4ab2 SUCCESS
1285 bc34ee8c371efc5c60d35d080da2b505e761ab0b1e5e68e1c46351faecedabc1 SUCCESS
1286 78d5c2cc7b6ebc22d8555bb2843ab8d8251d151555f41ae73295178462704baa SUCCESS
1287 a9324d2feecfd55f6a1b0f67ac63788489c8dbfed3c987007c1495a47e511bd5 SUCCESS
1288 ea981a9cc87fd3f248a3ccf70aa4ebba81de53df796a9a01706c29695238efb3 SUCCESS
1289 f18e2d9a4b8a9d5934f9e5c6f913ba10c5f4b34e53309b2810ad4317db577ce6 SUCCESS
1290 7ce3b990315ef199581e5f50556f40673df5829ad8cd76408f4abbb6921a5dc7 SUCCESS
1291 02ba204460f392adc21ca420fd1d56506f3a9e958139e8aa79d3ff08f60eadda SUCCESS
1292 70fbeba47e594aa68d478e5121eebbc4c3e25323bdb95e6fdc539a51f0e96974 SUCCESS
1293 f16f7d3af0597444ec1759e1ab14040c3731ec18c22d3d31fa078b7f51ba21fe SUCCESS
1294 3322e03b6eb92db1666988efe5bfade3d13dab4483138b7bb70c8798d7c

1388 1910af7f9c20836c46256a17a60c9b7ebded0b0dad15109afe3b49233bdeeb1b SUCCESS
1389 6773f2f3646611b456d1e62f03af6cf8f7add230bbe3a67c6f4a416af6d4c0c1 SUCCESS
1390 bd62619b512756ec93bd5a00a3388fabf492524f5caa6e4ec512f80db51c3f34 SUCCESS
1391 337dda8a1368078fab0f9215a9aa4e6ec92977a2604a2be93a5b873032692c97 SUCCESS
1392 cdb0311c397b945d8137f048e281185281c67cde869469fd101ea75818e21ada SUCCESS
1393 12987bd9682b24520049e7bab3f37ace5e1fe3ab6130e3956b4ee1f771674ffb SUCCESS
1394 ca28df1a98dc88f3cf906e5aed9a3cbeb0a25e9b4a0fc8d9a526100c0422caf6 SUCCESS
1395 7c3221eda29e8dd14113068b001843e2661d4f7d3d2d07ac23da80ca88e6942e SUCCESS
1396 fd42e5b29115e3d050f91edb02898eb135328832ca25ea83b9eaa655a10c046c SUCCESS
1397 43e42d59205aa164a037e995c4a4762f02348c8b312193d6ef124cc722869edc SUCCESS
1398 eeb5434686b3bb8594f9de9b7f95ac612a71689eebd3ef9de45740395411e55a SUCCESS
1399 2f0835d06e6b328f77f281f7e521c645f1a6fa348f48907e722db55f8bdcbaf7 SUCCESS
1400 b63c73b8956c70028651ed32cbb0ab066b8975662a3124c8fa360483c2e

In [17]:
session = requests.Session()
session.auth = (user, password)
print(session.get(MAAS_ENDPOINT + "available_results").text)

[
  {
    "config": {
      "config": {
        "run_id": "fc46f7f1606b1132512814a08e7420da69e0bc27543ab0a6be245b374dff25ad"
      },
      "name": "population_model"
    },
    "output": "https://s3.amazonaws.com/world-modelers/results/population_model/fc46f7f1606b1132512814a08e7420da69e0bc27543ab0a6be245b374dff25ad.csv",
    "status": "SUCCESS"
  },
  {
    "config": {
      "config": {
        "percent_of_normal_rainfall": 1,
        "run_id": "33878c54dc45056bb2c6b83fd71c9293a128ca1f45186a85aa976d76ded1dc89"
      },
      "name": "malnutrition_model"
    },
    "output": "https://s3.amazonaws.com/world-modelers/results/malnutrition_model/33878c54dc45056bb2c6b83fd71c9293a128ca1f45186a85aa976d76ded1dc89.geojson",
    "status": "SUCCESS"
  },
  {
    "config": {
      "config": {
        "percent_of_normal_rainfall": 80,
        "run_id": "22e92d6484b3d7baf0590140d30d8fc2945c6548e83e2271fe76fbbddd6f00a2"
      },
      "name": "malnutrition_model"
    },
    "output": "https://s3.ama

In [14]:
import wget
import zipfile
import os

def prep_p(session,model_name,runs,fields):
    
    dfs = {}
    out = open("chirps_runs.json","w")
    for run in runs:
        mydata = {
           "config":{},
           "name":model_name
        }
        for i, field in enumerate(fields):
            mydata["config"][field] = run[i]

        URL = MAAS_ENDPOINT + "run_model"
        print(json.dumps(mydata))
        resp = session.post(URL, json=mydata)
        rid = resp.text.replace('"',"").strip()
        dfs[rid] = mydata
        run_response = session.get(MAAS_ENDPOINT + "run_status/" + rid)
        STATUS = run_response.text.replace('"',"").strip()
    out.write(json.dumps(dfs,indent=2))
    out.close()
#         if STATUS == "SUCCESS":
#             URL = MAAS_ENDPOINT + "run_results"
#             run_response = session.get(MAAS_ENDPOINT + "run_results/" + rid)
#             outloc = json.loads(run_response.text)["output"]
#             wget.download(outloc,"./runs/")
            
            

In [15]:
session = requests.Session()
session.auth = (user, password)
prep_p(session,"CHIRPS",mmatrix,["_type","year","dekad"])

{"config": {"_type": "mm_data", "year": 2019, "dekad": "01"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2019, "dekad": "02"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2019, "dekad": "03"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2019, "dekad": "04"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2019, "dekad": "05"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2019, "dekad": "06"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2019, "dekad": "07"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2019, "dekad": "08"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2019, "dekad": "09"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2019, "dekad": "10"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2019, "dekad": "11"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2019, "dekad": "12"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 

{"config": {"_type": "mm_data", "year": 2016, "dekad": "01"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2016, "dekad": "02"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2016, "dekad": "03"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2016, "dekad": "04"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2016, "dekad": "05"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2016, "dekad": "06"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2016, "dekad": "07"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2016, "dekad": "08"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2016, "dekad": "09"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2016, "dekad": "10"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2016, "dekad": "11"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2016, "dekad": "12"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 

{"config": {"_type": "mm_data", "year": 2014, "dekad": "32"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2014, "dekad": "33"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2014, "dekad": "34"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2014, "dekad": "35"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2014, "dekad": "36"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2013, "dekad": "01"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2013, "dekad": "02"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2013, "dekad": "03"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2013, "dekad": "04"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2013, "dekad": "05"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2013, "dekad": "06"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2013, "dekad": "07"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 

{"config": {"_type": "mm_data", "year": 2011, "dekad": "33"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2011, "dekad": "34"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2011, "dekad": "35"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2011, "dekad": "36"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2010, "dekad": "01"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2010, "dekad": "02"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2010, "dekad": "03"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2010, "dekad": "04"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2010, "dekad": "05"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2010, "dekad": "06"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2010, "dekad": "07"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2010, "dekad": "08"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 

{"config": {"_type": "mm_data", "year": 2008, "dekad": "34"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2008, "dekad": "35"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2008, "dekad": "36"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2007, "dekad": "01"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2007, "dekad": "02"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2007, "dekad": "03"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2007, "dekad": "04"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2007, "dekad": "05"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2007, "dekad": "06"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2007, "dekad": "07"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2007, "dekad": "08"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 2007, "dekad": "09"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 

...

{"config": {"_type": "mm_data", "year": 1993, "dekad": "24"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1993, "dekad": "25"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1993, "dekad": "26"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1993, "dekad": "27"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1993, "dekad": "28"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1993, "dekad": "29"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1993, "dekad": "30"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1993, "dekad": "31"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1993, "dekad": "32"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1993, "dekad": "33"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1993, "dekad": "34"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1993, "dekad": "35"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 

{"config": {"_type": "mm_data", "year": 1990, "dekad": "21"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1990, "dekad": "22"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1990, "dekad": "23"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1990, "dekad": "24"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1990, "dekad": "25"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1990, "dekad": "26"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1990, "dekad": "27"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1990, "dekad": "28"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1990, "dekad": "29"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1990, "dekad": "30"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1990, "dekad": "31"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1990, "dekad": "32"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 

{"config": {"_type": "mm_data", "year": 1987, "dekad": "16"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1987, "dekad": "17"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1987, "dekad": "18"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1987, "dekad": "19"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1987, "dekad": "20"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1987, "dekad": "21"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1987, "dekad": "22"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1987, "dekad": "23"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1987, "dekad": "24"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1987, "dekad": "25"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1987, "dekad": "26"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1987, "dekad": "27"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 

{"config": {"_type": "mm_data", "year": 1984, "dekad": "13"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1984, "dekad": "14"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1984, "dekad": "15"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1984, "dekad": "16"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1984, "dekad": "17"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1984, "dekad": "18"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1984, "dekad": "19"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1984, "dekad": "20"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1984, "dekad": "21"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1984, "dekad": "22"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1984, "dekad": "23"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1984, "dekad": "24"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 

{"config": {"_type": "mm_data", "year": 1981, "dekad": "11"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1981, "dekad": "12"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1981, "dekad": "13"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1981, "dekad": "14"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1981, "dekad": "15"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1981, "dekad": "16"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1981, "dekad": "17"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1981, "dekad": "18"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1981, "dekad": "19"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1981, "dekad": "20"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1981, "dekad": "21"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 1981, "dekad": "22"}, "name": "CHIRPS"}
{"config": {"_type": "mm_data", "year": 